In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np

import math
import matplotlib.pyplot as plt

Для  градієнта нам потрібно розраховувати часткові похідні багатовимірної функції . Щоразу перераховувати,доволі енергозатратна затія. Чому  б під час розрахунку функції не "підготуватися" до пошуку похідних? Можна піти наступним шляхом: під час виконання певних разрахунків результат буде "пам'ятати" звідки він взявся,таким чином за топологічним сортуванням можна буде відтворити прохід графа,і рахувати похідні від "хвоста" до початку.

In [3]:
class Parameter:#цей клас нам потрібен для запам'ятовування крім числа та назви "сусідів у графі"
    def __init__(self, value: float, name: str, _parents=()) -> None:#
        self._value = value#кладемо числове значення
        self._name = name#кладемо ім'я
        self._parent=set(_parents)# зберігаємо попередників
        self._grad = 0.0# градіент спочатку нульовий
        self._backward = lambda: None#

    def __repr__(self) -> str:#для гарного print
        return f"Parameter {self._name} = {self._value}; dL/d[{self._name}] = {self._grad}"

    def __mul__(self, other) : #множення для нашого класу
        result = Parameter(
            self._value * other._value,#перемножимо числові значення
            f'{self._name} * {other._name}',#вивели на екран
            _parents=(self, other)#запам'ятали попередників
        )

        def _backward():# зворотній хід для множення
            self._grad += other._value * result._grad #dL / dself
            other._grad += self._value * result._grad # dL / dother

        result._backward = _backward

        return result

    def __add__(self, other):#додавання для нашого класу
        result = Parameter(
            self._value + other._value,#додаємо числові значення
            f'[{self._name} + {other._name}]',#вивели на екран
            _parents=(self, other)#запам'ятали попередників
        )

        def _backward():# зворотній хід для додавання
            self._grad += 1.0 * result._grad  #dL / dself
            other._grad += 1.0 * result._grad # dL / dother

        result._backward = _backward

        return result

    def sigmoid(self):
        # f(x) = 1 / (1 + exp(self._value))
        # f'(x) = f(x) * (1 - f(x))

        val = 1.0 / (1.0 + math.exp(-self._value))

        result = Parameter(
            val,
            f"σ({self._name})"
        )

        def _backward():
            self._grad = result._grad * val * (1 - val)

        result._backward = _backward

        return result
#     def ACT1TASK2(self):
#         # LAB 2 TASK 2: YOUR CODE GOES HERE (feel free to  rename)
    def backward(self):
        # LAB 2 TASK 1: YOUR CODE GOES HERE
        sorted=[]#список відсортованих вершин(масив ,а не множина,бо нас цікавить порядок)
        already_was=set()#множині не важливий порядок,а наявність вершин які вже задіяні
        
        def order(point):#функція організації наших вершин
            if point not in already_was:#якщо вершина не була задіяна
                already_was.add(point)#додаємо її до множини використаних
                for pair in point._parent:#для кожної пари "батьків"
                    order(pair)#відшукуємо їх "батьків"
                sorted.append(point)#додаємо до відсортованих
        order(self)#використовуємо функцію впорядкування до вершини
        
        self._grad = 1.0#в кінець кладемо одиницю
        for point in reversed(sorted):#розвертаємось від "хвоста до голови"
            point._backward()#до кожної вершини використвуємо зворотній хід


#     def ACT2TASK2(self):
#         # LAB 2 TASK 2: YOUR CODE GOES HERE (feel free to  rename)

    def silu(self):#функція активації сілу
        value = self._value * (1.0 / (1.0 + math.exp(-self._value)))#рахуємо значення
        result = Parameter(#створюємо змінну нашого класу 
            value,#кладемо значення
            f"silu({self._name})",
            _parents=(self,)#з батьків та сама змінна
        )

        def _backward():# градієнт що був домножуємо на похідну сілу
            self._grad += result._grad * ((1.0 / (1.0 + math.exp(-self._value))) * (1 + self._value * (1 - (1.0 / (1.0 + math.exp(-self._value))))))

        result._backward = _backward#присвоїли отримане результату

        return result#повернули результат
    #####################
    def elu(self, alpha=1.0):#функція активації сілу
        value = np.where(self._value > 0, self._value, alpha * (np.exp(self._value) - 1))#рахуємо значення
        result = Parameter(#створюємо змінну нашого класу 
            value,#кладемо значення
            f"elu({self._name})",
            _parents=(self,)#з батьків та сама змінна
        )

        def _backward():# градієнт що був домножуємо на похідну сілу
            self._grad += result._grad * (np.where(self._value > 0, 1, alpha * np.exp(self._value)))

        result._backward = _backward#присвоїли отримане результату

        return result#повернули результат
    
# def sgd(...) -> ...:
#     # LAB 2 TASK 3: YOUR CODE GOES HERE
def gd(learning_rate: float, *parameters: Parameter) -> None:
    for parameter in parameters:
        parameter._value -= learning_rate * parameter._grad
        parameter._grad = 0

In [4]:
a = Parameter(3.0, 'a')
b = Parameter(2.0, 'b')
c = Parameter(5.0, 'c')
d = Parameter(5.0, 'd')
u = a * b
v = u + c
L = v * d
L.backward()
print(L)
print(v)
print(u)
print(d)
print(c)
print(b)
print(a)

Parameter [a * b + c] * d = 55.0; dL/d[[a * b + c] * d] = 1.0
Parameter [a * b + c] = 11.0; dL/d[[a * b + c]] = 5.0
Parameter a * b = 6.0; dL/d[a * b] = 5.0
Parameter d = 5.0; dL/d[d] = 11.0
Parameter c = 5.0; dL/d[c] = 5.0
Parameter b = 2.0; dL/d[b] = 15.0
Parameter a = 3.0; dL/d[a] = 10.0


на прикладі з пари перевірено. Працює вірно

In [5]:
w = Parameter(0.5, 'w') #ваги котрі підбираємо
b = Parameter(0.1, 'b') # параметр б для підбору
x = Parameter(0.8, 'x')# вхідні дані
learning_rate = 0.01#крок навчання
n_epochs = 10#кількість епох(разів навчання)
goal = 0.3#ціль отримати такий результат
for n in range(n_epochs):
    y = (w * x + b).silu() #
    gd(learning_rate, w, b)
    loss = (y._value - goal) ** 2
    y.backward()
    print(f"loss after {n} epochs :{loss}")
print(w)#підібрані ваги
print(b)#підібраний б
print(x)#незмінний х
print((w * x + b).silu())#отримане значення

loss after 0 epochs :0.00012610538950865008
loss after 1 epochs :0.00012610538950865008
loss after 2 epochs :5.209908345990227e-06
loss after 3 epochs :4.355012617147169e-05
loss after 4 epochs :0.00023566931295804734
loss after 5 epochs :0.0005747995370237178
loss after 6 epochs :0.0010543518397158694
loss after 7 epochs :0.001667917663340409
loss after 8 epochs :0.0024092675865644195
loss after 9 epochs :0.00327234989206483
Parameter w = 0.447928485542994; dL/d[w] = 0.5575937337166031
Parameter b = 0.034910606928742496; dL/d[b] = 0.6969921671457538
Parameter x = 0.8; dL/d[x] = 3.416042721933838
Parameter silu([w * x + b]) = 0.23479809099209545; dL/d[silu([w * x + b])] = 0.0
